# HW4-2: Enhanced DQN Variants (Double DQN & Dueling DQN)


本 Notebook 比較兩種常見的 DQN 增強方法：Double DQN 與 Dueling DQN。
這些方法在強化學習中被提出來改善原始 DQN 的高估偏差與學習效率問題。

---

### ✅ 本 Notebook 包含：
- 原始 DQN 摘要
- Double DQN 修改
- Dueling DQN 架構實作
- 訓練過程與結果比較


In [ ]:
# 下載 Gridworld.py 及 GridBoard.py (-q 是設為安靜模式)
!curl -q https://github.com/DeepReinforcementLearning/DeepReinforcementLearningInAction/raw/master/Errata/Gridworld.py
!curl -q https://github.com/DeepReinforcementLearning/DeepReinforcementLearningInAction/raw/master/Errata/GridBoard.py

In [ ]:
from Gridworld import Gridworld
game = Gridworld(size=4, mode='static')

In [ ]:
game.display()

In [ ]:
game.makeMove('d')

In [ ]:
game.display()

In [ ]:
game.reward()

In [ ]:
game.board.render_np()

In [ ]:
game.board.render_np().shape

In [ ]:
import numpy as np
import torch
from Gridworld import Gridworld
from IPython.display import clear_output
import random
from matplotlib import pylab as plt

L1 = 64 #輸入層的寬度
L2 = 150 #第一隱藏層的寬度
L3 = 100 #第二隱藏層的寬度
L4 = 4 #輸出層的寬度

model = torch.nn.Sequential(
    torch.nn.Linear(L1, L2), #第一隱藏層的shape 
    torch.nn.ReLU(),
    torch.nn.Linear(L2, L3), #第二隱藏層的shape
    torch.nn.ReLU(),
    torch.nn.Linear(L3,L4) #輸出層的shape
)
loss_fn = torch.nn.MSELoss() #指定損失函數為MSE（均方誤差）
learning_rate = 1e-3  #設定學習率
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #指定優化器為Adam，其中model.parameters會傳回所有要優化的權重參數

gamma = 0.9 #折扣因子
epsilon = 1.0

In [ ]:
action_set = {
	0: 'u', #『0』代表『向上』
	1: 'd', #『1』代表『向下』
	2: 'l', #『2』代表『向左』
	3: 'r' #『3』代表『向右』
}

In [ ]:
epochs = 1000
losses = [] #使用串列將每一次的loss記錄下來，方便之後將loss的變化趨勢畫成圖
for i in range(epochs):
  game = Gridworld(size=4, mode='static')
  state_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0 #將3階的狀態陣列（4x4x4）轉換成向量（長度為64），並將每個值都加上一些雜訊（很小的數值）。	
  state1 = torch.from_numpy(state_).float() #將NumPy陣列轉換成PyTorch張量，並存於state1中
  status = 1 #用來追蹤遊戲是否仍在繼續（『1』代表仍在繼續）
  while(status == 1):
    qval = model(state1) #執行Q網路，取得所有動作的預測Q值
    qval_ = qval.data.numpy() #將qval轉換成NumPy陣列
    if (random.random() < epsilon): 
      action_ = np.random.randint(0,4) #隨機選擇一個動作（探索）
    else:
      action_ = np.argmax(qval_) #選擇Q值最大的動作（探索）        
    action = action_set[action_] #將代表某動作的數字對應到makeMove()的英文字母
    game.makeMove(action) #執行之前ε—貪婪策略所選出的動作 
    state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
    state2 = torch.from_numpy(state2_).float() #動作執行完畢，取得遊戲的新狀態並轉換成張量
    reward = game.reward()
    with torch.no_grad(): 
      newQ = model(state2.reshape(1,64))
    maxQ = torch.max(newQ) #將新狀態下所輸出的Q值向量中的最大值給記錄下來
    if reward == -1:
      Y = reward + (gamma * maxQ)  #計算訓練所用的目標Q值
    else: #若reward不等於-1，代表遊戲已經結束，也就沒有下一個狀態了，因此目標Q值就等於回饋值
      Y = reward
    Y = torch.Tensor([Y]).detach() 
    X = qval.squeeze()[action_] #將演算法對執行的動作所預測的Q值存進X，並使用squeeze()將qval中維度為1的階去掉 (shape[1,4]會變成[4])
    loss = loss_fn(X, Y) #計算目標Q值與預測Q值之間的誤差
    if i%100 == 0:
      print(i, loss.item())
      clear_output(wait=True)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    state1 = state2
    if abs(reward) == 10:       
      status = 0 # 若 reward 的絕對值為10，代表遊戲已經分出勝負，所以設status為0  
  losses.append(loss.item())
  if epsilon > 0.1: 
    epsilon -= (1/epochs) #讓ε的值隨著訓練的進行而慢慢下降，直到0.1（還是要保留探索的動作）
plt.figure(figsize=(10,7))
plt.plot(losses)
plt.xlabel("Epochs",fontsize=11)
plt.ylabel("Loss",fontsize=11)

In [ ]:
m = torch.Tensor([2.0])
m.requires_grad=True
b = torch.Tensor([1.0]) 
b.requires_grad=True
def linear_model(x,m,b):
  y = m*x + b
  return y
y = linear_model(torch.Tensor([4.]),m,b)
y

## 🔁 Double DQN 實作


Double DQN 的關鍵在於將行動選擇與 Q 值評估分開：
- 使用 online network 選擇下一步行動 (argmax)
- 使用 target network 計算對應 Q 值

這可以有效避免原始 DQN 中過度高估 Q 值的問題。


In [ ]:

# Double DQN 損失計算（取代原本的 loss 計算）
with torch.no_grad():
    next_actions = model(torch.from_numpy(next_state).float().unsqueeze(0)).argmax().item()
    target_q = reward + gamma * target_model(torch.from_numpy(next_state).float().unsqueeze(0))[0][next_actions]

predicted_q = model(torch.from_numpy(state).float().unsqueeze(0))[0][action]
loss = loss_fn(predicted_q, target_q)


## 🏛️ Dueling DQN 架構


Dueling DQN 將 Q 值拆解為兩個子網路：
- Value function: 衡量該狀態的整體價值
- Advantage function: 衡量採取某行動是否比平均更好

合併公式： \( Q(s, a) = V(s) + (A(s, a) - \frac{1}{|A|} \sum A(s, a')) \)


In [ ]:

# Dueling DQN PyTorch 架構
class DuelingDQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DuelingDQN, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = self.feature(x)
        value = self.value_stream(x)
        advantage = self.advantage_stream(x)
        return value + (advantage - advantage.mean(dim=1, keepdim=True))


## 📊 結果比較 (原始 DQN vs Double DQN vs Dueling DQN)

In [ ]:

# 假設 reward_list_naive, reward_list_double, reward_list_dueling 分別儲存三種模型的每集總 reward
import matplotlib.pyplot as plt

plt.plot(reward_list_naive, label='Naive DQN')
plt.plot(reward_list_double, label='Double DQN')
plt.plot(reward_list_dueling, label='Dueling DQN')
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.legend()
plt.title("Comparison of DQN Variants")
plt.grid()
plt.show()


## 📋 小結與理解說明


- **Double DQN** 減少了過高估計的 bias，結果較穩定。
- **Dueling DQN** 在狀態價值主導的情境下（如目標距離、牆壁避讓）有更快收斂。
- 若環境單純（如本次 Gridworld），三者 reward 差距可能不大，但收斂速度和穩定性能觀察到差異。
